# Introduciton and table of content for the notebook
## Load data
## Final Model
## ect

In [12]:
library(data.table,verbose = F,quietly = T)
library(mixOmics,verbose = F,quietly = T)
library(raster,verbose = F,quietly = T)
library(pracma,verbose = F,quietly = T)
library(ggpubr,verbose = F,quietly = T)
library(gplots,verbose = F,quietly = T)
library(RColorBrewer,verbose = F,quietly = T)
library(mediation,verbose = F,quietly = T)
library(cowplot,verbose = F,quietly = T)
library(caret,verbose = F,quietly = T)
library(randomForest,verbose = F,quietly = T)
library(ROCR,verbose = F,quietly = T)
library(caTools,verbose = F,quietly = T)

# Load HM450 data downloaded from GEO

In [13]:
sample_id_phase1 <- read.csv("./GEO/E-GEOD-80417.sdrf.txt",sep = "\t")
sample_id_phase2 <- read.csv("./GEO/E-GEOD-84727.sdrf.txt",sep = "\t")
phase1 <- fread("./GEO/GSE80417_normalizedBetas.csv", header = T)
phase2 <- fread("./GEO/GSE84727_normalisedBetas.csv", header = T)

# Remove Tobacco smoking associated probes from data

In [14]:
tobacco_schizo <- c("cg05575921",
                    "cg01940273",
                    "cg26703534",
                    "cg03636183",
                    "cg05951221",
                    "cg25952192",
                    "cg03274391",
                    "cg01772743",
                    "cg13862853",
                    "cg20566897")

In [15]:
common_probes <- intersect(phase1$V1,phase2$V1)
common_probes <- setdiff(common_probes,tobacco_schizo)

phase1 <- subset(phase1, V1 %in% common_probes)
phase2 <- subset(phase2, V1 %in% common_probes)

# Load Blood Cell Compositions, Smoking Scores, PRS

In [ ]:
blood_brain_pearson <- read.table("./Blood_Brain_Pearson_CoRR.txt",header = T)

In [17]:
Aberdeen_covariates <- read.csv("./data_sources/AdditionalCovariatesAberdeen.csv")
UCL_covariates <- read.csv("./data_sources/AdditionalCovariatesUCL.csv")
UCL_PRS <- read.csv("./data_sources/UCL_PRS.csv")
Aber_PRS <- read.csv("./data_sources/Aber_PRS.csv")

In [ ]:
model1 <-read.csv("./model_variables/model_corsiv.csv")
model2 <-read.csv("./model_variables/model_range.csv")
selected_variables <- rbind(model1,model2)
selected_variables  <- selected_variables[!(duplicated(selected_variables$probe_id)),]

In [ ]:
phase1 <- subset(phase1, V1 %in% selected_variables$probe_id)
phase2 <- subset(phase2, V1 %in% selected_variables$probe_id)

In [ ]:
colnames_phase1 <- phase1$V1
phase1$V1 <- NULL
phase1_t <- data.frame(t(phase1))
colnames(phase1_t) <- colnames_phase1
phase1_t$sample.id <- rownames(phase1_t)
rownames(phase1_t) <- NULL

In [ ]:
colnames_phase2 <- phase2$V1
phase2$V1 <- NULL
phase2_t <- data.frame(t(phase2))
colnames(phase2_t) <- colnames_phase2
phase2_t$sample.id <- rownames(phase2_t)
rownames(phase2_t) <- NULL

In [ ]:
#phase 2 data to train and phase 1 data to test
test <- data.frame(merge(sample_id_phase1,phase1_t,by.x = "id",by.y = "sample.id"))
train <- data.frame(merge(sample_id_phase2,phase2_t,by.x = "id",by.y = "sample.id"))

In [ ]:
head(Aberdeen_covariates)
head(UCL_covariates)
Aberdeen_covariates$cloz <- NULL
UCL_covariates$cloz <- NULL


In [ ]:
train <- merge(train,Aberdeen_covariates,by.x = "id",by.y="Basename")
test <- merge(test,UCL_covariates,by.x = "id",by.y = "Basename")

In [ ]:
#build on phase 2
train$CaseControl <- NULL
train$id <- NULL
train$sample_title <- NULL
train$status <- as.factor(ifelse(train$status==1,0,1))

test$CaseControl <- NULL
test$id  <- NULL
test$gender <- NULL
test$age <- NULL
test$status <- as.factor(ifelse(test$status==1,0,1))

In [ ]:
dim(train)
head(train)

dim(test)
head(test)

In [ ]:
X <- train[,2:235]
Y <- train$status

In [ ]:
list.keepX <- seq(1,230,10)
tune.splsda.final <- tune.splsda(X, Y, ncomp =6 , validation = 'Mfold', folds = 5, 
                           progressBar = TRUE, dist = 'max.dist', measure = "AUC",
                          test.keepX = list.keepX, nrepeat = 10, cpus = 6)
error <- tune.splsda.final$error.rate  # error rate per component for the keepX grid
ncomp <- tune.splsda.final$choice.ncomp$ncomp # optimal number of components based on t-tests
ncomp

In [ ]:
select.keepX <- tune.splsda.final$choice.keepX[1:ncomp]  # optimal number of variables to select
select.keepX
plot(tune.splsda.final, col = color.jet(6))

In [ ]:
splsda_output_beta <- mixOmics::splsda(X = subset(train,select = -c(status)),
                                        Y = train$status,ncomp = 3,keepX=c(50,50,50))

In [ ]:
# with background
background = background.predict(splsda_output_beta, comp.predicted=2, dist = "max.dist") 
#optional: xlim = c(-40,40), ylim = c(-30,30))

plotIndiv(splsda_output_beta, comp = 1:2,
          group = as.factor(train$status), title = "Maximum distance",
          legend = TRUE,  background = background)

In [ ]:
v <- plotIndiv(splsda_output_beta, legend=TRUE,comp = c(1,3),
               ellipse = T, star = FALSE, title = paste0('sPLS-DA Dimentions Plot-',"Schizophrenia"))
#confidence level set to 95%

In [ ]:
set.seed(40) # for reproducibility, only when the `cpus' argument is not used
# takes about 1 min to run
perf.srbct <- perf(splsda_output_beta, validation = "Mfold", folds = 5,
                   dist = 'max.dist', nrepeat = 10,
                   progressBar = FALSE) 
plot(perf.srbct, col = color.mixo(2))

par(mfrow=c(1,3))
plot(perf.srbct$features$stable[[1]], type = 'h', ylab = 'Stability', 
     xlab = 'Features', main = 'Comp 1', las =2)
plot(perf.srbct$features$stable[[2]], type = 'h', ylab = 'Stability', 
     xlab = 'Features', main = 'Comp 2', las =2)
plot(perf.srbct$features$stable[[3]], type = 'h', ylab = 'Stability', 
     xlab = 'Features', main = 'Comp 2', las =2)

# here we match the selected variables to the stable features
ind.match = match(selectVar(splsda_output_beta, comp = 1)$name, 
                  names(perf.srbct$features$stable[[1]]))
#extract the frequency of selection of those selected variables
Freq = as.numeric(perf.srbct$features$stable[[1]][ind.match])


In [ ]:
plotLoadings(splsda_output_beta, comp = 1, title = 'Loadings on comp 1', 
             contrib = 'max', method = 'mean')
plotLoadings(splsda_output_beta, comp = 2, title = 'Loadings on comp 2', 
             contrib = 'max', method = 'mean')
plotLoadings(splsda_output_beta, comp = 3, title = 'Loadings on comp 3', 
             contrib = 'max', method = 'mean')

In [ ]:
cim(splsda_output_beta,comp=1, title ="Component 1")
cim(splsda_output_beta,comp=2, title ="Component 2")
cim(splsda_output_beta,comp=3, title ="Component 3")

In [ ]:
plotVar(splsda_output_beta)

In [ ]:
plsdaDF <- as.data.frame(v$df)
highest_x_y <- ceiling(max(max(abs(plsdaDF$x)),max(abs(plsdaDF$y))))
marginal_plot(x = x, y = y, group = group, data = v$df,
              bw = "nrd", lm_formula = NULL, xlab = "sPLS-DA Dim 1", ylab = "sPLS-DA Dim 2", pch = 16, cex =1.5)

In [ ]:
var(plsdaDF$x)
var(plsdaDF$y)
rad2deg(atan2(2,1))

In [ ]:
# plsdaDF <- as.data.frame(v$df)
highest_x_y <- ceiling(max(max(abs(plsdaDF$x)),max(abs(plsdaDF$y))))
risk_distance <- function(cancer_sample){
    X <- as.numeric(cancer_sample[1])
    Y <- as.numeric(cancer_sample[2])
    d <- pointDistance(c(0,0),c(X,Y),lonlat = F)
    return(d*cos(atan2(Y,X) + deg2rad(180-rad2deg(atan2(3,1)))))# - for MBCN + for BC
    #return(d*cos(atan2(Y,X) + deg2rad(45)))# - for MBCN + for BC

}
plsdaDF$risk_distance <- -1*apply(plsdaDF, 1, risk_distance)
plot.data <- plsdaDF[c("group","risk_distance")]
cutoff <- 2*sd(plot.data[plot.data$group==0,]$risk_distance)
ggdensity(plot.data, x = "risk_distance",
          rug = TRUE,
          color = "group", fill = "group",
          palette = c("blue", "red"))
stdev_controls <- sd(plot.data[plot.data$group==0,]$risk_distance)#increase n
mean_controls <- mean(plot.data[plot.data$group==0,]$risk_distance)
print(stdev_controls)
print(mean_controls)

In [ ]:
Y <- test$status

In [ ]:
test <- test[colnames(splsda_output_beta$X)]

In [ ]:
########### PREDICTION ######
##########################################################################

#test$status <- NULL
test.predict <- predict(splsda_output_beta, test)

predicted_RD <- data.frame(-1*apply(test.predict$variates,1,risk_distance))
predicted_RD$Y <- as.numeric(as.character(Y))
colnames(predicted_RD) <- c("predictedRD","Y")

predicted_RD$Y <- as.factor(predicted_RD$Y)
ggdensity(predicted_RD, x = "predictedRD",
          rug = TRUE,
          color = "Y", fill = "Y",
          palette = c("blue", "red"))

In [ ]:
predicted_RD$catagory <- ifelse(predicted_RD$Y==0,"Negative","Positive")
colnames(plot.data) <- c("catagory","predictedRD")

final_RD_table <- rbind(
    predicted_RD[c("predictedRD","catagory")],
    plot.data[c("predictedRD","catagory")]
)
    
ggdensity(final_RD_table, x = "predictedRD",
          rug = TRUE,
          color = "catagory", fill = "catagory",
          palette = c("blue", "red","yellow","green"))

In [ ]:
#################################################
PPV <-c()
num_of_predicted_ind <- c()
#stdev <- sd(plot.data[plot.data$catagory==0,]$risk_distance)#increase n
#mean_PRD <- mean(plot.data[plot.data$catagory==0,]$risk_distance)
n_controls <- c()
n_cases <- c()


for(n in seq(1,3,0.5)){
    cutoff <- n*stdev_controls
    cutoff <- mean_controls + cutoff
    
    predicted_RD_cutoff <- predicted_RD[predicted_RD$predictedRD>cutoff,]
    #n_controls_in_sd <- dim(predicted_RD_cutoff[predicted_RD_cutoff$Y==0,])[1]
    #n_cases_in_sd <- dim(predicted_RD_cutoff[predicted_RD_cutoff$Y==1,])[1]
    predicted_RD_cutoff$predicted_Y <- as.numeric(ifelse(predicted_RD_cutoff$predictedRD>cutoff,1,0))
    
    PPV <- c(PPV,
             dim(predicted_RD_cutoff[predicted_RD_cutoff$Y==1 & predicted_RD_cutoff$predicted_Y==1 ,])[1]/dim(predicted_RD_cutoff[predicted_RD_cutoff$predicted_Y==1 ,])[1])
    num_of_predicted_ind <- c(num_of_predicted_ind,dim(predicted_RD_cutoff)[1])
}

plot(x=seq(1,3,0.5),y=round(PPV,2),type="o",col="red",xlab="Number of standard deviations from mean")
plot(x=seq(1,3,0.5),y=num_of_predicted_ind,type="o",col="blue",xlab="Number of standard deviations from mean",ylab="Number of individuals predicted")

In [ ]:
num_of_predicted_ind
PPV

In [ ]:
varComp1 <- selectVar(splsda_output_beta,comp = 1)
varComp2 <- selectVar(splsda_output_beta,comp = 2)

selected_C1 <- data.frame(varComp1$value)
selected_C2 <- data.frame(varComp2$value)

selected_C1$probe_id <- row.names(selected_C1)
selected_C2$probe_id <- row.names(selected_C2)

In [ ]:
varible_importance <- rbind(selected_C1[order(abs(selected_C1$value.var),decreasing=T),]
,selected_C2[order(abs(selected_C2$value.var),decreasing=T),])

varible_importance$value.var <- abs(varible_importance$value.var)
varible_importance <- varible_importance[order(varible_importance$value.var,decreasing = T),]

In [ ]:
selected_var <- as.character(varible_importance$probe_id)

In [ ]:
selected_var <- c('status',unique(selected_var))

In [ ]:
test <- data.frame(merge(sample_id_phase1,phase1_t,by.x = "id",by.y = "sample.id"))
train <- data.frame(merge(sample_id_phase2,phase2_t,by.x = "id",by.y = "sample.id"))
train <- merge(train,Aberdeen_covariates,by.x = "id",by.y="Basename")
test <- merge(test,UCL_covariates,by.x = "id",by.y = "Basename")

In [ ]:
train <- train[selected_var]
test <- test[selected_var]

In [ ]:
rf_model=randomForest(as.factor(status) ~.,data = train,ntree=500,mtry=50,importance=TRUE)
prediction <- predict(rf_model,subset(test,select = -c(status)))
table(observed = test$status,predicted=prediction)
# Calculate the probability of new observations belonging to each class
# prediction_for_roc_curve will be a matrix with dimensions data_set_size x number_of_classes
prediction_for_roc_curve <- predict(rf_model,subset(test,select = -c(status)),type="prob")
# Use pretty colours:
pretty_colours <- c("#F8766D")
# Specify the different classes 
classes <- levels(as.factor(test$status))
# For each class
i = 1
 # Define which observations belong to class[i]
true_values <- ifelse(test$status==classes[i],2,1)
pred <- prediction(prediction_for_roc_curve[,i],true_values)

perf <- performance(pred, "tpr", "fpr")
plot(perf,main="ROC Curve",col="#F8766D") 
auc.perf <- performance(pred, measure = "auc")
auc.perf@y.values

In [ ]:
varImpPlot(rf_model,type = 2)

In [ ]:
head(blood_brain_pearson)

In [ ]:
data <- subset(blood_brain_pearson, probe_id %in% selected_var)
rownames(data) <- data$probe_id
data$probe_id <- NULL
options(repr.plot.width=15, repr.plot.height=10)
heatmap.2(as.matrix(t(data)),col=rev(brewer.pal(7,"RdBu")),main = "Blood-Brain Correlation Final Model Probes",
          trace="none",margins = c(8, 16),
          distfun = function(x) dist(x, method="manhattan"),
          hclustfun = function(x) hclust(x,method="average"),scale="none",key = FALSE)

In [ ]:
predicted_RD$Basename <- UCL_covariates$Basename

In [ ]:
UCL_PRS <- read.csv("./UCL_PRS.csv")


In [ ]:
predicted_RD <- merge(predicted_RD,UCL_PRS,by="Basename")

In [ ]:
cor.test(predicted_RD$predictedRD,predicted_RD$SCORE,method = "spearman")

In [ ]:
options(repr.plot.width=15, repr.plot.height=15)
sp <- ggscatter(predicted_RD, x = "SCORE", y = "predictedRD",
                color = "catagory", palette = c("blue","red"),
                size = 2, alpha = 0.6, ggtheme = theme_bw(base_size = 20))             
# Marginal boxplot of x (top panel) and y (right panel)
xplot <- ggboxplot(predicted_RD, x = "catagory", y = "SCORE", 
                   color = "catagory", fill = "catagory", palette = c("blue","red"),
                   alpha = 0.5, ggtheme = theme_bw())+
  rotate()
yplot <- ggboxplot(predicted_RD, x = "catagory", y = "predictedRD",
                   color = "catagory", fill = "catagory", palette = c("blue","red"),
                   alpha = 0.5, ggtheme = theme_bw())
# Cleaning the plots
sp <- sp + rremove("legend")
yplot <- yplot + clean_theme() + rremove("legend")
xplot <- xplot + clean_theme() + rremove("legend")
# Arranging the plot using cowplot

plot_grid(xplot, NULL, sp, yplot, ncol = 2, align = "hv", 
          rel_widths = c(2, 1), rel_heights = c(1, 2))


In [ ]:
head(predicted_RD)

In [ ]:


fit.totaleffect=glm(Y~SCORE,predicted_RD,family = "binomial")
summary(fit.totaleffect)

fit.mediator=lm(predictedRD~SCORE,data=predicted_RD)
summary(fit.mediator)

fit.dv=glm(Y~SCORE+predictedRD,data=predicted_RD,family = "binomial")
summary(fit.dv)

summary(mediate(fit.mediator, fit.dv, treat='SCORE', mediator='predictedRD',robustSE = TRUE, sims = 1000))